In [5]:
import datetime as dt
import json as js
import urllib.request

url='http://api.openweathermap.org/data/2.5/forecast?lat=64.286670&lon=27.622125&appid=e25afeccd6e22a2e996bed2809e43452'

resp=urllib.request.urlopen(url)
jsfile_Tornio=js.load(resp)

def date_predict(time_survey):
    time_survey=dt.datetime.strptime(time_survey,'%Y-%m-%d %H:%M:%S')
    return '{}-{:0>2d}-{:0>2d}'.format(time_survey.year,time_survey.month, time_survey.day)

date_temp_dictionary={}

for each_date in jsfile_Tornio['list']:
    datee=date_predict(each_date['dt_txt'])
    if datee not in date_temp_dictionary:
        date_temp_dictionary[datee]=[]
        date_temp_dictionary[datee].append(round(each_date['main']['temp']-273,1))
    else:
        date_temp_dictionary[datee].append(round(each_date['main']['temp']-273,1))

        
date_weather_info=[]
for i in date_temp_dictionary:
    if len(date_temp_dictionary[i])==8:
        temp_max=max(date_temp_dictionary[i])
        temp_min=min(date_temp_dictionary[i])
        temp_avg=sum(date_temp_dictionary[i])/len(date_temp_dictionary[i])
        
        date_weather_info.append(
        {'date':i,
        'temp_avg':temp_avg,
        'temp_max':temp_max,
        'temp_min':temp_min,
        'rain(mm)':0})
    else:
        pass

date_list=[i['date'] for i in date_weather_info]
for each_date in jsfile_Tornio['list']:
    datee=date_predict(each_date['dt_txt'])
    if datee in date_list:
        ind=date_list.index(datee)
        try:
            rain_amount=each_date['rain'] 
            # if there are no rain at the survey time, you can't find the 'rain' key in each_date, 
            # then the program will jump to the except state.
            if rain_amount>date_weather_info[ind]['rain(mm)']:
                date_weather_info[ind]['rain(mm)']=rain_amount
            else:
                pass

        except KeyError:
            # if there has been rained already on the same day,
            # then we do nothing to the rain information in date_weather_info
            pass
    else:
        continue

In [6]:
date_weather_info

[{'date': '2020-04-09',
  'temp_avg': 0.6375,
  'temp_max': 3.4,
  'temp_min': -4.4,
  'rain(mm)': 0},
 {'date': '2020-04-10',
  'temp_avg': -2.6750000000000003,
  'temp_max': 0.5,
  'temp_min': -5.7,
  'rain(mm)': 0},
 {'date': '2020-04-11',
  'temp_avg': -1.3250000000000002,
  'temp_max': 1.4,
  'temp_min': -3.4,
  'rain(mm)': 0},
 {'date': '2020-04-12',
  'temp_avg': -1.2249999999999999,
  'temp_max': 1.5,
  'temp_min': -3.6,
  'rain(mm)': 0},
 {'date': '2020-04-13',
  'temp_avg': -2.2750000000000004,
  'temp_max': -0.9,
  'temp_min': -4.6,
  'rain(mm)': 0}]